Для того, чтобы посчитать смены, следующий алгоритм

Есть соответсвие между операцией, станком, количеству ресурсов

Инфраструктура определяет, сколько одновременно можно сделать операций

И люди тоже привязаны к инфраструктуре

Поэтому надо сделать следующий файл с конфигурацией:

Станок Количество_станков Операция Тип_людей Количество_людей 

Останется только перевести из нормо-часов в дни.

1) Сначала заполняем по максимому дневные смены
2) при нехватке в месяц дневных смен, заполняем ночные 

Как будто это можно сделать средствами пандаса

1. Берём мерджим таблицу с операциями и таблицу конфигурации
2. Нормо-часы делим на количество станков
3. Дублируем каждую строчку со станком в соответсввии с количеством и дропаем количество
4. Получаем остаток от деления количества нормо-часов на 12, записываем это в колонку num_shifts
5. Делаем табличку со списком требуемых сотрудников
6. Прибавляем к num_shifts дату старта, тем самым получая с какого числа по какое будет дневная смена
7. Для тех, кто num_shifts > max_days, прибавляем  к дате старта (num_days - num_shift) (надо учесть, что может быть большой разрыв)

Этот алгоритм верен при параллельных операциях

На деле происходит последовательные операции. Глупо будет думать, что вот они сначала сделают все детали B, чтобы сделать деталь А. Наверное, сначала сделают n деталей В, потом их передадут дальше. По принципу конвейера, это просто даст некотурую начальную задерджку. Возмонжно, она будет в рамках часа, не более. Поэтому такой алгоритм может дать приблизительную оценку.

In [52]:
import pandas as pd
import numpy as np

In [53]:
staff_conf = pd.read_excel("../data/StaffConfig.xlsx")
times = pd.read_excel("../data/answ.xlsx").drop(columns=["Unnamed: 0"])

In [54]:
staff_conf

,Станок,Количество станков,Операция,Специалист,Количество людей
0,Лазерный станок 2030,1,Лазерная резка листа,Оператор станок с пу/лазер,1
1,Продольно-фрезерный станок - вертикальный,1,Вертикально-фрезерная,Станочник широкого профиля,5
2,Токарный станок (SLZ-500E),1,Токарная,Станочник широкого профиля,5
3,Пост сборки,1,Сборочная,Слесарь по сборке,2
4,Пост сборки,1,Упаковочная,Слесарь по сборке,1
5,Ленточная пила (MER Shark 281 CCS),1,Ленточно-отрезная,Слесарь по сборке,4
6,Дисковая пила (KKS 45 1NA),1,Слесарная,Слесарь по сборке,1
7,Сварочный пост,1,Сварка полуавтоматом в среде защитного газа (MIG),Эл. Сварщик и п/авт машин,1
8,Листосгибочный станок 7630,1,Листогибочная,Оператор станок с пу/гибка,1
9,Вальцовочный станок (Prinzng RME70/103),1,Вальцовочнуя,Оператор станок с пу/гибка,1


In [55]:
times

,Operation,Time
0,Лазерная резка листа,9.159444
1,Листогибочная,44.241111
2,Окрашивание порошком,8.000000
3,Сборочная,20.750000
4,Сварка полуавтоматом в среде защитного газа (MIG),38.166667
5,Упаковочная,0.833333


In [56]:
merged_times = (staff_conf.rename(columns={"Операция" : "Operation"})
               .merge(times, on=["Operation"], how="right"))
merged_times

,Станок,Количество станков,Operation,Специалист,Количество людей,Time
0,Лазерный станок 2030,1,Лазерная резка листа,Оператор станок с пу/лазер,1,9.159444
1,Листосгибочный станок 7630,1,Листогибочная,Оператор станок с пу/гибка,1,44.241111
2,Лента порошковой окраски,1,Окрашивание порошком,Оператор окрасочно-сушильной линии и агрегата,1,8.000000
3,Лента порошковой окраски,1,Окрашивание порошком,Рабочий по металлопрокату и окраске,1,8.000000
4,Пост сборки,1,Сборочная,Слесарь по сборке,2,20.750000
5,Сварочный пост,1,Сварка полуавтоматом в среде защитного газа (MIG),Эл. Сварщик и п/авт машин,1,38.166667
6,Пост сборки,1,Упаковочная,Слесарь по сборке,1,0.833333


In [57]:
unknown_operations = merged_times[merged_times["Станок"].isna()]["Operation"].to_list()
unknown_operations

[]

In [58]:
#merged_times["Количество станков"] = 2

In [59]:
merged_times["time_per_machine"] = merged_times["Time"] / merged_times["Количество станков"]
merged_times = merged_times.loc[np.repeat(merged_times.index, merged_times["Количество станков"])]

In [60]:
merged_times

,Станок,Количество станков,Operation,Специалист,Количество людей,Time,time_per_machine
0,Лазерный станок 2030,1,Лазерная резка листа,Оператор станок с пу/лазер,1,9.159444,9.159444
1,Листосгибочный станок 7630,1,Листогибочная,Оператор станок с пу/гибка,1,44.241111,44.241111
2,Лента порошковой окраски,1,Окрашивание порошком,Оператор окрасочно-сушильной линии и агрегата,1,8.000000,8.000000
3,Лента порошковой окраски,1,Окрашивание порошком,Рабочий по металлопрокату и окраске,1,8.000000,8.000000
4,Пост сборки,1,Сборочная,Слесарь по сборке,2,20.750000,20.750000
5,Сварочный пост,1,Сварка полуавтоматом в среде защитного газа (MIG),Эл. Сварщик и п/авт машин,1,38.166667,38.166667
6,Пост сборки,1,Упаковочная,Слесарь по сборке,1,0.833333,0.833333


In [ ]:
max_days: int =  10

merged_times["num_shifts"] = np.ceil(merged_times["Time"] / 12)
merged_times["num_max_days"] = np.floor(merged_times["num_shifts"] / max_days)
merged_times["num_night_shift"] = merged_times["num_max_days"] // 2 * max_days  + (merged_times["num_shifts"] - merged_times["num_max_days"] * max_days) * (merged_times["num_max_days"] % 2)
merged_times["num_day_shift"] = merged_times["num_shifts"] - merged_times["num_night_shift"]

In [ ]:
a = lambda num_max_days, max_days, num_shifts: np.floor(num_max_days)  // 2 * max_days  + (num_shifts - np.floor(num_max_days) * max_days) * (np.floor(num_max_days) % 2)

for i,j,k in [(0.8, 10, 8), (1.1, 10, 11), (1.2, 10,12), (2.2, 10, 22), (2.3, 10, 23), (3.2, 10, 32)]:
    print(f"{i,j,k}->{a(i,j,k)}")

(0.8, 10, 8)->0.0
(1.1, 10, 11)->1.0
(1.2, 10, 12)->2.0
(2.2, 10, 22)->10.0
(2.3, 10, 23)->10.0
(3.2, 10, 32)->12.0


In [ ]:
(np.floor(1.2) % 2) * 10  + (32 - np.floor(3.2) * 10) * (np.floor(3.2) % 2)

np.float64(22.0)

In [90]:
np.floor(3.2) // 2

np.float64(1.0)

In [83]:
(np.floor(3.2) % 2) * 10

np.float64(10.0)

In [96]:
prepared_staff = merged_times[["Специалист", "Количество людей", "num_day_shift", "num_night_shift"]]
prepared_staff

,Специалист,Количество людей,num_day_shift,num_night_shift
0,Оператор станок с пу/лазер,1,1.0,0.0
1,Оператор станок с пу/гибка,1,4.0,0.0
2,Оператор окрасочно-сушильной линии и агрегата,1,1.0,0.0
3,Рабочий по металлопрокату и окраске,1,1.0,0.0
4,Слесарь по сборке,2,2.0,0.0
5,Эл. Сварщик и п/авт машин,1,4.0,0.0
6,Слесарь по сборке,1,1.0,0.0


In [26]:
import datetime
import pandas as pd

In [31]:
operations_params: dict[str, list] = {"Сотрудник": ["Programmer"],
                                      "Операция": ["Coding"],
                                      "Количество": [1]}

operations_dates: dict[str, list[tuple[datetime.date, bool]]] = {"Programmer|Coding":
[(datetime.date(2024,12,12), False, 9.9), (datetime.date(2024,12,12), True, 9.1), (datetime.date(2024,12,13), False, 5)]}

min_date = datetime.date(2024,12,12)
max_date = datetime.date(2024,12,13)

In [30]:
staff_table = pd.DataFrame(operations_params)
staff_table

,Сотрудник,Операция,Количество
0,Programmer,Coding,1


In [29]:
(max_date - min_date).days

1

In [32]:
base_range = [(min_date + datetime.timedelta(days=i)).strftime("%d-%M-%Y") for i in range((max_date - min_date).days + 1)]
columns: list = [[date + " День", date + " Ночь"] \
                      for date in base_range]
columns = [item for row in columns for item in row]
dates = pd.DataFrame(columns=columns, dtype=float)
dates

,12-00-2024 День,12-00-2024 Ночь,13-00-2024 День,13-00-2024 Ночь


In [33]:
merged = pd.concat([staff_table, dates]).fillna(0)

In [34]:
new_op_dates = {}

for val, dates in operations_dates.items():
    tmp = []
    for date, is_night, count in dates:
        if is_night:
            tmp.append((date.strftime("%d-%M-%Y") + " Ночь", count))
        else:
            tmp.append((date.strftime("%d-%M-%Y") + " День", count))
    new_op_dates[val] = tmp
new_op_dates

{'Programmer|Coding': [('12-00-2024 День', 9.9),
  ('12-00-2024 Ночь', 9.1),
  ('13-00-2024 День', 5)]}

In [29]:
(merged["Сотрудник"] == val.split("|")[0]) & (merged["Операция"] == val.split("|")[1])

0    True
dtype: bool

In [35]:
for val, dates in new_op_dates.items():
    dates_ = [date[0] for date in dates]
    count = [count[1] for count in dates]
    merged.loc[(merged["Сотрудник"] == val.split("|")[0]) &
               (merged["Операция"] == val.split("|")[1]), dates_] = count

In [36]:

merged

,Сотрудник,Операция,Количество,12-00-2024 День,12-00-2024 Ночь,13-00-2024 День,13-00-2024 Ночь
0,Programmer,Coding,1.0,9.9,9.1,5.0,0.0


In [38]:
merged.style.format(precision=2)

,Сотрудник,Операция,Количество,12-00-2024 День,12-00-2024 Ночь,13-00-2024 День,13-00-2024 Ночь
0,Programmer,Coding,1.00,9.90,9.10,5.00,0.00


In [ ]:
import datetime
datetime.date(year=2024, day=12,month=12).

,Сотрудник,Операция,Количество,12-00-2024 День,12-00-2024 Ночь,13-00-2024 День,13-00-2024 Ночь
0,Programmer,Coding,1.0,Да,Да,Да,Нет


In [22]:
def color_survived(val):
    color = "default"
    if val == 'Да':
        color = 'green'
    if val == 'Нет':
        color = 'red'
    return f'background-color: {color}'

merged.style.applymap(color_survived)

C:\Users\ввв\AppData\Local\Temp\ipykernel_5416\28707094.py:9: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  merged.style.applymap(color_survived)


,Сотрудник,Операция,Количество,12-00-2024 День,12-00-2024 Ночь,13-00-2024 День,13-00-2024 Ночь
0,Programmer,Coding,1.000000,Нет,Нет,Нет,Нет
